# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
前章で作成したマルコフモデルの様な「ある単語から次の単語を予測するモデル」をニューラルネットワークを用いて作成する。

本章では、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import random
from typing import List

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### データセット

wiki40b

In [3]:
textfile = 'data/jawiki.txt'
with open(textfile) as f:
    data = f.readlines()

print('num of data:', len(data))
data[:3] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。\n',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。\n']

多すぎるので減らす

In [4]:
n_data = 2000
data = data[:n_data]

textfile = f'data/jawiki_{n_data}.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence)


---

## サブワード分割

トークン化の手法。

NNでも、マルコフモデル同様、文章をトークン化して学習を行う。  
NNでは文字列をそのまま扱えないので、各トークンにID=クラスラベルを割り当てて、そのシーケンスとして文章を扱う。

前章とは異なり、サブワード分割という新たな手法を使う。ちゃんと理由もある。

1トークンあたりの文字数が多いことは、いくつかのデメリットを生む。  
例えば、トークンの種類が多くなること。深層学習を用いた言語モデルはトークンの種類に比例してパラメータの数が増え、学習が困難になる。  
それから、未知語が増えること。長いトークンは、それだけ多くの情報を持った限定的な言葉ということとなり、これらで語彙が埋まると表現力が落ちる。またデータセットに存在しない言葉を扱うことが困難になる。

サブワード分割は、これらの問題を解決する。  
サブワードと呼ばれる、単語よりも小さな単位に分割することで、1トークンあたりの文字数を減らす。

この手法では、データセットから頻出する文字の並びを学習し、その並びをトークンとして分割する。学習された、トークン化を行うモデルをトークナイザと呼ぶ。  
データセットに合ったトークン化が可能。また語彙数を指定することも可能。指定した語彙数に収まるまで細かく分割してくれる。

ちなみに、ChatGPTが使っているトークナイザはこれ: [OpenAI Platform](https://platform.openai.com/tokenizer)

少し余談。

前章でトークン化について以下のように説明した。

> トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。

ここで、そもそも言語モデルは単語の確率を扱うものなので、絶対にトークン=単語にならないとおかしいとも考えられる。  
実は文章生成においては、単語を最小単位にしなければならない理由はない。極端な話、文字を最小単位として文章を文字の並びと見てもいい訳だ。なんらかのシーケンスとできればそれで充分なのだ。実際、現在有名な言語モデルの多くは単語を最小単位としていない。

ただ、言語モデルは単語の並びに確率を割り当てるモデルだ。単語を最小単位とするモデルだ。  
もしこの定義に厳格になるのであれば、現在有名な多くの言語モデルは言語モデルと呼べないのかもしれんね。しらんけど。

閑話休題。実際にやってみよう。  
sentencepieceというライブラリを用いる。

In [5]:
import sentencepiece as spm

### 学習

語彙数とデータセット（テキストファイル）を指定して、学習させる。

In [ ]:
tokenizer_prefix = f'models/tokenizer_jawiki_{n_data}' # トークナイザのモデル名
vocab_size = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=textfile, # データセット
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size
)

### トークン化

学習したモデルにテキストを突っ込むとトークン化してくれる。

In [7]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model') # モデル読み込み
text = '今日はいい天気だ'
ids = sp.encode(text)
ids

[11, 1897, 6, 2436, 664, 287, 346]

テキストがトークン化され、ID列として取得できる。  
`out_type`を指定すると文字列のリストが取得できる。

In [8]:
sp.encode(text, out_type=str)

['▁', '今日', 'は', 'いい', '天', '気', 'だ']

アンダーバーは空白を意味する。初めにアンダーバーが付くのは仕様。

ID列を文字列に戻すこともできる。

In [9]:
sp.decode(ids)

'今日はいい天気だ'

全てのデータをトークン化（ID化）しよう。  
文章のリスト: `List[str]`を`sp.encode()`に与えるとID列のリスト: `List[List[int]]`が返ってくる。

In [10]:
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 19, 477, 653, 323, 51, 570, 57, 3856, 1583]


---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### *Unknown*

未知語を意味するトークン。  
推論時に学習データに含まれなかった単語に出会ったときに対応できるようになる。学習データ内での出現回数が少ない単語も未知語として扱うことがある。

### *Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### *End of Sentence*

文章の終わりを意味するトークン。  
先程の例では句点が出た場合に生成を止めたが、本来句点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが直接触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けたりすることが多い。

sentencepieceでは上記3つのトークンがID0, 1, 2にデフォルトで設定されている。

In [11]:
print('ID0:', sp.id_to_piece(0)) # Unknown
print('ID1:', sp.id_to_piece(1)) # Begin of Sentence
print('ID2:', sp.id_to_piece(2)) # End of Sentence

ID0: <unk>
ID1: <s>
ID2: </s>


では、トークン列にBOSとEOSを追加して学習させてみよう。

In [12]:
bos_id = sp.bos_id() # BOSのID
eos_id = sp.eos_id() # EOSのID
for ids in data_ids:
    ids.insert(0, bos_id) # 先頭にBOSを追加
    ids.append(eos_id) # 末尾にEOSを追加


---

## 学習データの作成

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [13]:
Ids = List[int]
class TextDataset(Dataset):
    def __init__(self, data_ids: List[Ids]):
        self.x = [] # 入力
        self.t = [] # 正解
        for ids in data_ids:
            self.x += ids[:-1]
            self.t += ids[1:]
        self.n_data = len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]

    def __len__(self):
        return self.n_data

dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 64
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)
test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)

# examples
x, y = next(iter(train_loader))
x, y

(tensor([  73,   13,  154,  495,   11,   71, 1750, 5906,  794,   36, 1383,  104,
         1916,   52, 2798,  209, 5429,   11, 4401,   31,    5, 1387, 1278,  240,
         2610, 4529,    3, 2491,    1,  782, 2460, 2642,    3, 2249,  321,  357,
           12, 5065,  428, 1306,    4,  399, 3134,  286,  213,   94,   11,  116,
            5,   30,    8,   61,  841,  606,   53,   11, 2032,   14, 1501,  179,
           36,  110,   34,    4]),
 tensor([  31,   40,    5,  240,  252,   68, 4520,    9, 2958,    5, 3080,    7,
          583, 1643,    3,  188,    6, 1521,  827, 3813, 2503, 1378,  840,    7,
           26,    3,   84,  493,   11, 2077,    5,  589, 4005, 1690, 1531,   38,
          150,  829, 4262,  126, 4088,    3,   18,    9,   39,    4, 2506, 1535,
         1428, 1863,  459,   23,    6,  125, 1901, 5116,    8,    3,    9, 2278,
         1554, 1308,  215, 5676]))


---

## 埋め込み層

*Embedding Layer*

指定したIDに対応するベクトルを出力する層。

単語IDはクラスラベルなので、そのままNNに入力するのは適切ではない。そこで、埋め込み層と呼ばれる層を用いて単語IDを指定した次元のベクトルに変換する。単語をベクトル化することは埋め込みと呼ばれるので、埋め込み層と呼ばれる。

埋め込み層は語彙の数だけベクトルを持っている。IDが入力されると、対応するベクトルが出力される。  
実装してみよう。

In [14]:
class Embedding(nn.Module):
    def __init__(self, n_vocab: int, embed_dim: int):
        super().__init__()
        self.vectors = torch.nn.Parameter(torch.randn(n_vocab, embed_dim))

    def forward(self, x):
        h = self.vectors[x]
        return h

embedding = Embedding(3, 5)

x = torch.tensor([0, 1, 2])
h = embedding(x)
h

tensor([[ 1.4508,  2.4973,  0.0191, -0.4259,  0.7090],
        [ 0.6346,  1.5298,  0.2450,  0.9936,  0.7267],
        [-2.0840,  0.9692,  0.8315,  0.0060, -0.0687]],
       grad_fn=<IndexBackward0>)

`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
- https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

In [15]:
embedding = torch.nn.Embedding(3, 5)
h = embedding(x)
h

tensor([[ 0.8123,  0.9160,  0.2652, -0.3974, -0.2158],
        [-0.4996, -2.3171,  1.9133,  0.9837, -0.8494],
        [ 1.3092,  0.8357, -1.4483,  0.1485,  0.4845]],
       grad_fn=<EmbeddingBackward0>)

埋め込み層が持っているベクトルは学習可能なパラメータである。  
埋め込み層がone-hot化+線形変換と同じことを行っていると見ればパラメータであることが理解しやすい。

言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させる試みもあり、word2vecなどがこれに該当する。


---

## モデル構築

ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

実際の出力は語彙数分の次元を持つベクトルとなる。  
つまりこのタスクは単語の分類問題とも言える。

In [16]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Embedding(n_vocab, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x):
        """
        x: (batch_size,)
        """
        y = self.net(x) # (batch_size, n_vocab)
        return y


---

## 学習

損失関数に交差エントロピーを設定し、通常の分類モデルと同じように学習する。

In [17]:
loss_fn = nn.CrossEntropyLoss()

def eval_model(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, t in test_loader:
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = loss_fn(y, t)
            total_loss += loss.item()
    loss = total_loss / len(test_loader)
    return loss

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            optimizer.zero_grad()
            x = x.to(device) # 入力
            t = t.to(device) # 正解
            y = model(x) # 出力
            loss = loss_fn(y, t) # 損失
            loss.backward() # 逆伝播
            optimizer.step() # パラメータ更新
            prog.update(loss.item())
        test_loss = eval_model(model)
        prog.memo(f'test: {test_loss:.5f}')

In [18]:
n_vocab = len(sp)
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [19]:
train(model, optimizer, n_epochs=20, prog_unit=2)

  1-2/20: ############################## 100% [00:00:49.24] loss train: 6.58968, test: 6.42639 
  3-4/20: ############################## 100% [00:00:45.75] loss train: 5.68028, test: 6.26719 
  5-6/20: ############################## 100% [00:00:45.46] loss train: 5.29898, test: 6.22287 
  7-8/20: ############################## 100% [00:00:44.78] loss train: 5.07391, test: 6.21966 
 9-10/20: ############################## 100% [00:00:43.56] loss train: 4.92347, test: 6.23964 
11-12/20: ############################## 100% [00:00:44.44] loss train: 4.81598, test: 6.26655 
13-14/20: ############################## 100% [00:00:43.97] loss train: 4.73577, test: 6.29763 
15-16/20: ############################## 100% [00:00:45.18] loss train: 4.67358, test: 6.33519 
17-18/20: ############################## 100% [00:00:44.53] loss train: 4.62425, test: 6.37206 
19-20/20: ############################## 100% [00:00:44.48] loss train: 4.58395, test: 6.41051 


テストデータでの精度は低くなるのはしょうがないかな。  
直前の単語のみを見て次の単語を予測しているので、初見の文章を正確に予測するのは無理。


---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。マルコフモデル同様、あるトークンからの次のトークンの予測を繰り返すことで文章を生成する。

初めのトークンはBOSとし、以下の条件を満たすまで単語の生成を続ける。
- EOSが生成される
- 単語数が指定した限度に達する

In [20]:
unk_id = sp.unk_id() # UNKのID

# モデルの出力から単語をサンプリングする関数
def token_sampling(y: List[float]) -> int:
    y[unk_id] = -torch.inf # UNKがサンプリングされる確率を0にする
    probs = F.softmax(y, dim=-1) # 確率分布に変換
    token, = random.choices(range(n_vocab), weights=probs) # サンプリング
    return token

# 文章を生成する関数
def generate_sentence(model, max_len=50):
    model.eval()
    token_id = bos_id
    token_ids = []

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != eos_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        token_id = token_sampling(y) # サンプリング
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [21]:
for _ in range(5):
    print(generate_sentence(model))

ゲーリングのVSEa」とりにも、これにわずか10軍のパラは御山の内容にせ、また、ブルックリンやを終え、死因果たの中間されることが多い。170合作者を付け頭髪などの出ていた。
祭りたて屋根付き広場大学院侯の微博としいぶ。司法永14:)。
8%の英雄は、甥、1961世のア(Sを発行に輝いた。これについてはメディデラーとともに「チンランドと初の6月22年ロンドンは「PRed Hopず、2010年7
木造に向か、アシスタントコーチを務めている牛の息子として中納する「STSF1世率いるデンマーク王寺であり、ナチ党の未知れられている。ハウスのギッキー・形で田小学校司教職業、さを装
:-音頭痛はラーシュテープ - 1つの大型点から収録されてしまったの中等のコンクリートの初めにしたサルチョの助手く、多摩市立て、草飛行士から正明和也


マルコフモデル同様、直前の単語のみを予測に用いているため、不自然な文章が多く生成される。

出来上がったモデルは優れたものではないが、深層学習を用いた言語モデル実装の基礎を学ぶことができた。